In [1]:
from pathlib import Path # reads paths in the current OS
import pandas as pd
import numpy as np
import yaml
import utils as ut

In [2]:
with open(Path('conf') / 'paths.yaml') as file:
    config = yaml.full_load(file)

abastiment = pd.read_csv(Path(config['input_path']) / "abastiment_clean.csv", sep=",").fillna('')
pagesos    = pd.read_csv(Path(config['input_path']) / "pagesos_clean.csv", sep=",").fillna('')

In [3]:
abastiment['comarca_origin'].unique()

array(['Osona', 'Cerdanya', "Ribera d'Ebre", 'Barcelonès',
       'Baix Llobregat', 'Maresme', 'Garrigues', 'Tarragonès', 'Bages',
       "Vall d'Aran", 'Conca de Barberà', 'Pallars Jussà', 'Segarra',
       'Baix Camp', 'Selva', 'Pallars Sobirà', 'Terra Alta', 'Moianès',
       'Alt Empordà', 'Vallès Oriental', 'Noguera', '(NOTFOUND)',
       'Baix Penedès', 'Gironès', 'Berguedà', 'Anoia', 'Alt Urgell',
       'Vallès Occidental', 'Priorat', 'Baix Empordà', 'Alt Camp',
       'Baix Ebre', "Pla d'Urgell", 'Alt Penedès', 'Solsonès',
       'Alta Ribagorça', 'Ripollès', 'Garrotxa', 'Segrià', 'Urgell',
       'Montsià', 'Garraf'], dtype=object)

In [4]:
abastiment.rename(columns={'PROJECTE':'MARCA'},inplace=True)
pagesos['dataset']    = 'pagesos'
abastiment['dataset'] = 'abastiment'

In [5]:
matches = pagesos['MARCA'].apply(lambda row: ut.run_project_match(row, abastiment['MARCA']))
matches = matches.str.split(';',expand=True).rename(columns={0:'exact',1:'MARCA'})

Partial match: BdeGust Beer Cervesa Compromesa vs. BdeGust Cervesa Compromesa 91 81
Partial match: BIOGRASSFEED vs. Biograssfed 96 91
Partial match: Tomating vs. Tomàting 88 88
Partial match: Ramaderia Ecològica ca l'Andreu vs. Ramaderia ecológica ca l’andreu 94 94
Partial match: L'Enllaç de l'Ebre vs. L'Enllaç de l’Ebre, SCCL 79 89
Partial match: (Mu)DA Vedella d'Agullana vs. MuDA. Vedella d'Agullana 94 96
Partial match: Granja de Montmelús vs. Granja i formatgeria de Montmelús 73 84
Partial match: Safra del Montsec vs. Safrà del Montsec 94 94
Partial match: Celler Optim vs. Celler Lo 76 89
Partial match: FORMATGERIA LAURA vs. Formatgeria Tiraval 78 82
Partial match: Cordú Fruits vs. Cor de fruita 80 83
Partial match: PLANESES AGRICULTIRA REGENERATIVA vs. Planeses Agricultura Regenerativa 97 97
Partial match: Granja Can Torrent vs. Granja can Borrell 83 83
Partial match: Maduixes del Maresme vs. Maduixes de Munt 72 81
Partial match: Can Terrades Produccio Agraria Ecològica sco vs. Ca

In [6]:
# manually checked for names that where filtered as a match but weren't
not_duplicates = ['Celler Lo','Formatgeria Tiraval','Cor de fruita','Granja can Borrell','Maduixes de Munt']

In [7]:
# keep the values that are duplicates only
matches = matches[matches.exact != '0']

In [8]:
# remove false positives
ind = matches[matches['MARCA'].str.contains('\b'+r'\b|\b'.join(not_duplicates)+'\b')].index
matches = matches.drop(ind)

In [9]:
common_cols = [c for c in pagesos.columns if c in abastiment.columns]

In [10]:
all_data = pd.concat([pagesos[common_cols],
          abastiment[~(abastiment.MARCA.isin(matches.MARCA.values))][common_cols]],
          axis=0)

In [11]:
all_data.to_csv(Path(config['input_path']) / 'all_data.csv', index=False)

In [12]:
all_data['comarca_origin'].unique()

array(['Baix Llobregat', "Ribera d'Ebre", 'Noguera', 'Vallès Oriental',
       'Alt Camp', 'Tarragonès', 'Selva', 'Segarra', 'Montsià',
       'Alt Empordà', 'Baix Penedès', "Pla d'Urgell", 'Baix Ebre',
       'Berguedà', 'Segrià', 'Terra Alta', 'Urgell', 'Conca de Barberà',
       'Osona', 'Alt Penedès', 'Ripollès', 'Anoia', 'Solsonès',
       'Pallars Jussà', 'Bages', 'Priorat', 'Garrigues', 'Baix Empordà',
       'Gironès', 'Baix Camp', 'Vallès Occidental', 'Pallars Sobirà',
       'Alt Urgell', 'Maresme', 'Garrotxa', 'Moianès', 'Alta Ribagorça',
       "Pla de l'Estany", '(NOTFOUND)', 'Garraf', 'Cerdanya',
       'Barcelonès', "Vall d'Aran"], dtype=object)